In [ ]:
import argparse
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import math
import numpy as np
import pandas as pd
from pathlib import Path
import pandas as pd
import pickle
import re
import sys
from scipy import stats
from scipy.stats import mannwhitneyu
import matplotlib as mpl
import functools 

In [ ]:
import sys
## /lab/users/jhonDoe/Code/Get_cover_rust_pysam
sys.path.append("<path to coverage_rust_bam directory>")
## this is import the  python lib that uses the brust backend 
## see readme for install 
import Get_cover 

In [ ]:
def get_attr(string):
    dico = {}
    spt_ = string.split(";")
    
    for x in spt_:
        if x:
            try:
                dico[x.split()[0].strip()] =  x.split()[1].replace('"', "").strip()
            except:
                print(spt_)
                raise
    return dico


def gtf_to_dict(gtf_file, main_ = "gene_id"):
    print(main_)
    dico = {}
    with open(gtf_file) as f_in:
        """atm reads only genes"""
        for line in f_in:
            line = line.strip()
            if not line:
                continue
            if line.startswith("#"):
                continue
            spt = line.strip().split("\t")
            try:
                chr_ = spt[0]
                start = int(spt[3]) 
                end = int(spt[4]) 
                strand =  spt[6]
                attr = get_attr(spt[-1])
                gene_id = attr[main_]
                gene_symbol = attr.get("gene_symbol", "None")
                type_ = spt[2]
            except:
                print(attr)
                print(line)
                print(spt)
                raise
            
            if gene_id not in dico:
                dico[gene_id] = {
                    "chr": chr_,
                    "symbol" :  gene_symbol,
                    "strand" : strand,
                    "transcript": {}
                }
                
            if type_ == "gene":
    
                dico[gene_id]["start"] = start
                dico[gene_id]["end"] = end
            
            else:
                try:
                    transcript_id  = attr.get("transcript_id", "None")
                    transcript_symbol = attr.get("transcript_symbol", "None")

                    
                    if transcript_id not in dico[gene_id]["transcript"]:
                        dico[gene_id]["transcript"][transcript_id] = {
                        "transcript_symbol" : transcript_symbol,
                        "transcript_id" : transcript_id
                        }
                    
                    if type_ not in dico[gene_id]["transcript"][transcript_id]: 
                        dico[gene_id]["transcript"][transcript_id][type_] = []
                        
                    dico[gene_id]["transcript"][transcript_id][type_].append({
                        "chr": chr_,
                        "start": start, 
                        "end" : end,
                        "strand" : strand,
                    })
                except: 
                    continue

In [ ]:
gtf_file = "<path to annotated Y genes gtf>"
gtf_dict = gtf_to_dict(gtf_file, "gene_id") # if you don't use the gtf we provides be sure there is a "gene_id" attributes
bam_dir = Path("<path to directory with bam files>")
# bam dir with alignment (of D.mauritiana and hybrid) against D. mauritiana inside all hybrids file start with trimmed_sim_mau
# optionaly give a list of the path of all bam files in bam_files variable

In [ ]:
bam_file = list(bam_dir.glob("*bam"))
bam_file.sort()

# check those bam_file are the one you want!
for bam in bam_file:
    print(bam)

# if not copy paste the one you want 
# bam_file = [... ,... , ...]

In [ ]:
#bam_file = [str(bam_dir /  x) for x in bams]
#{bam_file.sort()

In [ ]:
out_dir="<Path to outdir>"

In [ ]:
for g in gtf_dict:

    for transcript in gtf_dict[g]["transcript"]:
        intervalls = gtf_dict[g]["transcript"][transcript]["exon"]

        for e in intervalls:
            e["type"] = "exon"

        results = Get_cover._get_Coverage_intervall(bam_file, intervalls=intervalls, n_thread=8, lib_scheme="frFirstStrand", mapq=13, flag_in=0, flag_out=256)
        print("pool done")
        for bam in results:
            results[bam].max_reads = Get_cover.get_max_read(bam)
            results[bam].normalize()
        
        
        res = {}
        for file in bam_file:
            if "L001" in file:
                f2 = file.replace("L001", "L002")
                X = Path(file).stem
                res[X] = Get_cover.Coverage.merge_duplicates([results[file], results[f2]])
                                                        
                                                         
        c2 = ["#abd9e9","#74add1","#4575b4","#313695"] 
        c1 = ["#d73027","#f46d43","#fdae61","#fee090"]

        dico_colors = {}
        c1_cpt = 0
        c2_cpt = 0
        for bam in res:
            if "trimmed_sim_mau" in Path(bam).stem:
                dico_colors[bam] = c1[c1_cpt]
                c1_cpt += 1

            else:
                dico_colors[bam] = c2[c2_cpt]
                c2_cpt += 1

        dico_labels = dict((x, x) for x in  res)

        Get_cover.plot(res, N=None, intron="exon",
                       intron_prop=0.35,
                          dico_color=dico_colors,
                          title=gtf_dict[g]["symbol"]+" "+g,
                        out=out_dir + gtf_dict[g]["symbol"] + "_exon_mau.pdf",
                       linewidth=3)
        plt.show();